In [1]:
import sys
import numpy as np
def load_data(filename):
    num_users = 0
    num_items = 0
    user2id = {}
    item2id = {}
    for line in open(filename, "r"):
        items = line.strip().split("\t")
        userid = items[0]
        songid = items[1]
        if userid not in user2id:
            user2id[userid] = num_users
            num_users += 1
        if songid not in item2id:
            item2id[songid] = num_items
            num_items += 1
    matrix = np.zeros((num_users, num_items))
    for line in open(filename, "r"):
        items = line.strip().split("\t")
        user = items[0]
        song = items[1]
        userid = user2id[user]
        songid = item2id[song]
        matrix[userid, songid] = 1
    return matrix, item2id, user2id
matrix, item2id, user2id = load_data("./ratings.1m")

In [2]:
len(matrix)

17651

In [19]:
import sklearn
import time
from  numpy import random as rng
from sklearn.utils.extmath  import *
from scipy.special import expit
from sklearn.utils import check_random_state
from sklearn.utils import validation
from sklearn.utils.validation import check_is_fitted
from sklearn.utils import issparse
import scipy as sp
class RBM():
    def __init__(self, n_components=256, learning_rate=0.1, batch_size=10,
                 n_iter=10, verbose=0, random_state=None, iter_array=None):
        self.n_components = n_components
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.n_iter = n_iter
        self.verbose = verbose
        self.random_state = random_state  
        self.iter_array = iter_array
    def fit(self, X, y=None, ):
        """Fit the model to the data X.
        Parameters
        ----------
        X : {array-like, sparse matrix} shape (n_samples, n_features)
            Training data.
        Returns
        -------
        self : BernoulliRBM
            The fitted model.
        """
        X = check_array(X, accept_sparse='csr', dtype=np.float64)
        n_samples = X.shape[0]
        rng = check_random_state(self.random_state)

        #W初始化使用的是正太分布，均值是0，sigma 是0.01
        #W:(n_components, n_features)
        self.components_ = np.asarray(
            rng.normal(0, 0.01, (self.n_components, X.shape[1])),
            order='F')
        #hidden:(n_components)
        self.intercept_hidden_ = np.zeros(self.n_components, )
        #visible:(n_features)
        self.intercept_visible_ = np.zeros(X.shape[1], )
        #batch of hidden components
        self.h_samples_ = np.zeros((self.batch_size, self.n_components))

        n_batches = int(np.ceil(float(n_samples) / self.batch_size))
        batch_slices = list(sklearn.utils.gen_even_slices(n_batches * self.batch_size,
                                            n_batches, n_samples))
        verbose = self.verbose
        begin = time.time()
        if self.iter_array is  None:
            self.iter_array = np.array([self.n_iter])
        self.iter_array = np.sort(self.iter_array)
        self.n_iter = np.max(self.iter_array)
        likehood_array = []
        for iteration in xrange(1, self.n_iter + 1):
            for batch_slice in batch_slices:
                self._fit(X[batch_slice], rng)
            #
            #likehood = self.score_samples(X).mean()
            #if iteration in self.iter_array:
            #    likehood_array.append(likehood)
            
            if verbose:
                end = time.time()
                print("[%s] Iteration %d,log-likehood:%s"
                      " time = %.2fs"
                      % (type(self).__name__, iteration,
                        self.score_samples(X).mean() , end - begin))
                begin = end

        return self.iter_array, likehood_array   
    def rmse(self, v):
        h_pos = self._mean_hiddens(v)
        v_neg = self._sample_visibles(h_pos, rng)
        
        return np.sqrt(((v_neg-v)**2).mean())
    def _free_energy(self, v):
        """Computes the free energy F(v) = - log sum_h exp(-E(v,h)).
        Parameters
        ----------
        v : array-like, shape (n_samples, n_features)
            Values of the visible layer.
        Returns
        -------
        free_energy : array-like, shape (n_samples,)
            The value of the free energy.
        """
        return (- safe_sparse_dot(v, self.intercept_visible_)
                - np.logaddexp(0, safe_sparse_dot(v, self.components_.T)
                               + self.intercept_hidden_).sum(axis=1))    
    def score_samples(self, X):
        """Compute the pseudo-likelihood of X.
        Parameters
        ----------
        X : {array-like, sparse matrix} shape (n_samples, n_features)
            Values of the visible layer. Must be all-boolean (not checked).
        Returns
        -------
        pseudo_likelihood : array-like, shape (n_samples,)
            Value of the pseudo-likelihood (proxy for likelihood).
        Notes
        -----
        This method is not deterministic: it computes a quantity called the
        free energy on X, then on a randomly corrupted version of X, and
        returns the log of the logistic function of the difference.
        """
        check_is_fitted(self, "components_")
        v= sklearn.utils.check_array(X, accept_sparse='csr')
        rng = check_random_state(self.random_state)
        # Randomly corrupt one feature in each sample in v.
        ind = (np.arange(v.shape[0]),
               rng.randint(0, v.shape[1], v.shape[0]))
        if issparse(v):
            data = -2 * v[ind] + 1
            v_ = v + sp.csr_matrix((data.A.ravel(), ind), shape=v.shape)
        else:
            v_ = v.copy()
            v_[ind] = 1 - v_[ind]        

        fe = self._free_energy(v)
        fe_ = self._free_energy(v_)
        return v.shape[1] * sklearn.utils.extmath.log_logistic(fe_ - fe)    
    def _fit(self, v_pos, rng):
        """Inner fit for one mini-batch.
        Adjust the parameters to maximize the likelihood of v using
        Stochastic Maximum Likelihood (SML).
        Parameters
        ----------
        v_pos : array-like, shape (n_samples, n_features)
            The data to use for training.
        rng : RandomState
            Random number generator to use for sampling.
        """
        h_pos = self._mean_hiddens(v_pos)
        #not pcd is also ok for rating.1m
        h_samples = np.zeros(h_pos.shape)
        h_samples[rng.uniform(size=h_pos.shape) < h_pos] = 1.0
        #v_neg = self._sample_visibles(self.h_samples_, rng)
        #v_neg = self._sample_visibles(h_samples, rng)
        
        #try:v_neg use the prob, the result is also ok, maybe better than binary
        v_neg = self._mean_visibles(h_samples)
        h_neg = self._mean_hiddens(v_neg)

        lr = float(self.learning_rate) / v_pos.shape[0]
        #v_pos:binary, h_pos:prob
        #h_neg:prob, v_neg:prob
        update = safe_sparse_dot(v_pos.T, h_pos, dense_output=True).T
        update -= np.dot(h_neg.T, v_neg)
        self.components_ += lr * update
        self.intercept_hidden_ += lr * (h_pos.sum(axis=0) - h_neg.sum(axis=0))
        self.intercept_visible_ += lr * (np.asarray(
                                         v_pos.sum(axis=0)).squeeze() -
                                         v_neg.sum(axis=0))

        #h_neg[rng.uniform(size=h_neg.shape) < h_neg] = 1.0  # sample binomial
        #self.h_samples_ = np.floor(h_neg, h_neg)    
    def _mean_hiddens(self, v):
        """Computes the probabilities P(h=1|v).
        Parameters
        ----------
        v : array-like, shape (n_samples, n_features)
            Values of the visible layer.
        Returns
        -------
        h : array-like, shape (n_samples, n_components)
            Corresponding mean field values for the hidden layer.
        """
        p = safe_sparse_dot(v, self.components_.T)
        p += self.intercept_hidden_
        return expit(p, out=p)   
    def _mean_visibles(self, h):
        p = np.dot(h, self.components_)
        p += self.intercept_visible_
        return expit(p, out=p)        
    def _sample_visibles(self, h, rng):
        """Sample from the distribution P(v|h).
        Parameters
        ----------
        h : array-like, shape (n_samples, n_components)
            Values of the hidden layer to sample from.
        rng : RandomState
            Random number generator to use.
        Returns
        -------
        v : array-like, shape (n_samples, n_features)
            Values of the visible layer.
        """
        p = np.dot(h, self.components_)
        p += self.intercept_visible_
        expit(p, out=p)
        return (rng.random_sample(size=p.shape) < p)    


In [20]:
rbm=RBM(n_components=10, learning_rate=0.1, batch_size=10, n_iter=20, verbose=True,random_state=0)
rbm.fit(matrix)

[instance] Iteration 1,log-likehood:-452.412274341 time = 15.37s
[instance] Iteration 2,log-likehood:-440.65124482 time = 18.28s
[instance] Iteration 3,log-likehood:-431.333197693 time = 18.07s
[instance] Iteration 4,log-likehood:-423.787343675 time = 18.24s
[instance] Iteration 5,log-likehood:-414.317513095 time = 17.95s
[instance] Iteration 6,log-likehood:-405.466131503 time = 17.91s
[instance] Iteration 7,log-likehood:-396.593557221 time = 17.91s
[instance] Iteration 8,log-likehood:-394.039474818 time = 17.87s
[instance] Iteration 9,log-likehood:-389.771557899 time = 17.98s
[instance] Iteration 10,log-likehood:-386.037747699 time = 17.83s
[instance] Iteration 11,log-likehood:-380.140423334 time = 17.96s
[instance] Iteration 12,log-likehood:-375.881125942 time = 17.91s
[instance] Iteration 13,log-likehood:-373.256944253 time = 17.73s
[instance] Iteration 14,log-likehood:-369.666616739 time = 18.04s
[instance] Iteration 15,log-likehood:-363.270578957 time = 18.05s
[instance] Iteration

(array([20]), [])

In [21]:
weights = rbm.components_.T
out_f = open("song_vec.txt", "w")
for item in item2id:                       
    id = item2id[item]
    line = weights[id]
    #bias = visible_bias[id]
    out_f.write(str(item) + "\t" + "\t".join(map(str, line))+ "\n")
out_f.close()

In [ ]:
import pandas as pd
iter_array = [10, 50, 100,200]
latent_factors = [10, 50, 100]
regularizations = [0.1, 0.01, 0.001, 1]

best_params = {}
best_params['n_factors'] = latent_factors[0]
best_params['reg'] = regularizations[0]
best_params['n_iter'] = 0
best_params['mse'] = np.inf
best_params['model'] = None

for fact in latent_factors:
    print 'Factors: {}'.format(fact)
    for reg in regularizations:
        print 'Regularization: {}'.format(reg)
        rbm=RBM(n_components=fact, learning_rate=reg, batch_size=10, n_iter=50, verbose=True,random_state=0,iter_array = iter_array)
        sort_array, likehhood_array = rbm.fit(matrix) 
        min_idx = np.argmax(likehhood_array)
        if likehhood_array[min_idx] < best_params['mse']:
            best_params['n_factors'] = fact
            best_params['reg'] = reg 
            best_params['n_iter'] = sort_array[min_idx]
            best_params['mse'] = likehhood_array[min_idx]
            best_params['model'] = rbm
            print 'New optimal hyperparameters'
            print pd.Series(best_params)
            

Factors: 10
Regularization: 0.1
[instance] Iteration 1,log-likehood:-456.57081878 time = 27.13s
[instance] Iteration 2,log-likehood:-458.773277398 time = 26.77s
[instance] Iteration 3,log-likehood:-456.687196061 time = 26.30s
[instance] Iteration 4,log-likehood:-459.471524134 time = 26.24s
[instance] Iteration 5,log-likehood:-456.935035545 time = 26.56s
[instance] Iteration 6,log-likehood:-461.170939035 time = 26.13s
[instance] Iteration 7,log-likehood:-463.320878106 time = 26.51s
[instance] Iteration 8,log-likehood:-452.129796333 time = 26.44s
[instance] Iteration 9,log-likehood:-427.207747648 time = 26.68s
[instance] Iteration 10,log-likehood:-426.000706325 time = 26.27s
[instance] Iteration 11,log-likehood:-421.548547807 time = 26.96s
[instance] Iteration 12,log-likehood:-412.403007269 time = 26.57s
[instance] Iteration 13,log-likehood:-405.692393962 time = 25.83s
[instance] Iteration 14,log-likehood:-399.340395801 time = 26.84s
[instance] Iteration 15,log-likehood:-389.519732988 ti

[instance] Iteration 128,log-likehood:-326.669182595 time = 27.06s
[instance] Iteration 129,log-likehood:-329.221593736 time = 26.99s
[instance] Iteration 130,log-likehood:-335.455635691 time = 27.33s
[instance] Iteration 131,log-likehood:-332.731754924 time = 26.58s
[instance] Iteration 132,log-likehood:-321.834618159 time = 27.10s
[instance] Iteration 133,log-likehood:-327.661317074 time = 27.54s
[instance] Iteration 134,log-likehood:-323.632746933 time = 26.26s
[instance] Iteration 135,log-likehood:-323.060065962 time = 27.17s
[instance] Iteration 136,log-likehood:-338.495493089 time = 27.70s
[instance] Iteration 137,log-likehood:-328.844793067 time = 27.53s
[instance] Iteration 138,log-likehood:-325.400714111 time = 27.01s
[instance] Iteration 139,log-likehood:-326.695749149 time = 26.98s
[instance] Iteration 140,log-likehood:-333.972151223 time = 26.95s
[instance] Iteration 141,log-likehood:-335.824507517 time = 27.23s
[instance] Iteration 142,log-likehood:-325.106122983 time = 27

[instance] Iteration 49,log-likehood:-447.990902085 time = 26.91s
[instance] Iteration 50,log-likehood:-448.006832985 time = 27.03s
[instance] Iteration 51,log-likehood:-449.041646408 time = 27.65s
[instance] Iteration 52,log-likehood:-447.509012695 time = 26.76s
[instance] Iteration 53,log-likehood:-448.587991697 time = 26.31s
[instance] Iteration 54,log-likehood:-448.825818302 time = 27.33s
[instance] Iteration 55,log-likehood:-447.477201987 time = 26.70s
[instance] Iteration 56,log-likehood:-447.060540911 time = 28.03s
[instance] Iteration 57,log-likehood:-448.221927816 time = 28.43s
[instance] Iteration 58,log-likehood:-448.221026072 time = 27.76s
[instance] Iteration 59,log-likehood:-448.713712751 time = 27.57s
[instance] Iteration 60,log-likehood:-448.543732162 time = 27.07s
[instance] Iteration 61,log-likehood:-447.940504048 time = 26.41s
[instance] Iteration 62,log-likehood:-447.554420153 time = 26.03s
[instance] Iteration 63,log-likehood:-448.624003586 time = 26.81s
[instance]

[instance] Iteration 174,log-likehood:-448.10516908 time = 27.25s
[instance] Iteration 175,log-likehood:-447.093663283 time = 28.12s
[instance] Iteration 176,log-likehood:-447.370358802 time = 26.53s
[instance] Iteration 177,log-likehood:-448.11989685 time = 27.85s
[instance] Iteration 178,log-likehood:-447.999270165 time = 28.22s
[instance] Iteration 179,log-likehood:-447.985248753 time = 28.22s
[instance] Iteration 180,log-likehood:-447.101933614 time = 26.42s
[instance] Iteration 181,log-likehood:-447.702956747 time = 26.57s
[instance] Iteration 182,log-likehood:-447.904075198 time = 26.83s
[instance] Iteration 183,log-likehood:-447.558736857 time = 26.44s
[instance] Iteration 184,log-likehood:-446.850562182 time = 27.91s
[instance] Iteration 185,log-likehood:-446.550961116 time = 26.93s
[instance] Iteration 186,log-likehood:-446.941751953 time = 26.62s
[instance] Iteration 187,log-likehood:-446.419396947 time = 27.99s
[instance] Iteration 188,log-likehood:-447.527414874 time = 26.8

[instance] Iteration 93,log-likehood:-444.624080708 time = 27.96s
[instance] Iteration 94,log-likehood:-444.345058275 time = 30.16s
[instance] Iteration 95,log-likehood:-444.223996157 time = 26.31s
[instance] Iteration 96,log-likehood:-444.449467903 time = 25.43s
[instance] Iteration 97,log-likehood:-444.465954598 time = 28.47s
[instance] Iteration 98,log-likehood:-444.588046819 time = 28.66s
[instance] Iteration 99,log-likehood:-444.528066938 time = 27.37s
[instance] Iteration 100,log-likehood:-444.432281312 time = 29.79s
[instance] Iteration 101,log-likehood:-444.567023768 time = 29.25s
[instance] Iteration 102,log-likehood:-444.65217649 time = 24.91s
[instance] Iteration 103,log-likehood:-444.795819936 time = 26.08s
[instance] Iteration 104,log-likehood:-444.794771573 time = 29.54s
[instance] Iteration 105,log-likehood:-444.765343304 time = 27.68s
[instance] Iteration 106,log-likehood:-444.885832214 time = 31.06s
[instance] Iteration 107,log-likehood:-444.783330951 time = 28.95s
[in

...

[instance] Iteration 116,log-likehood:-447.933259627 time = 53.06s
[instance] Iteration 117,log-likehood:-451.719275228 time = 53.23s
[instance] Iteration 118,log-likehood:-449.892283635 time = 52.94s
[instance] Iteration 119,log-likehood:-448.845961202 time = 53.22s
[instance] Iteration 120,log-likehood:-447.780063501 time = 52.56s
[instance] Iteration 121,log-likehood:-450.273982792 time = 53.38s
[instance] Iteration 122,log-likehood:-449.9248265 time = 53.73s
[instance] Iteration 123,log-likehood:-451.108809672 time = 53.04s
[instance] Iteration 124,log-likehood:-447.735328771 time = 52.90s
[instance] Iteration 125,log-likehood:-448.67923002 time = 53.42s
[instance] Iteration 126,log-likehood:-455.240217442 time = 52.69s
[instance] Iteration 127,log-likehood:-448.766077124 time = 52.84s
[instance] Iteration 128,log-likehood:-449.549790637 time = 53.22s
[instance] Iteration 129,log-likehood:-451.011188584 time = 53.12s
[instance] Iteration 130,log-likehood:-448.135104425 time = 53.11

[instance] Iteration 43,log-likehood:-445.781439533 time = 53.17s
[instance] Iteration 44,log-likehood:-445.093915273 time = 53.13s
[instance] Iteration 45,log-likehood:-445.508915932 time = 52.99s
[instance] Iteration 46,log-likehood:-445.987193771 time = 53.14s
[instance] Iteration 47,log-likehood:-446.721663003 time = 53.26s
[instance] Iteration 48,log-likehood:-445.890019675 time = 52.73s
[instance] Iteration 49,log-likehood:-446.153557132 time = 53.52s
[instance] Iteration 50,log-likehood:-445.897745851 time = 53.16s
[instance] Iteration 51,log-likehood:-445.551346546 time = 52.94s
[instance] Iteration 52,log-likehood:-446.508860324 time = 52.95s
[instance] Iteration 53,log-likehood:-446.206113677 time = 52.86s
[instance] Iteration 54,log-likehood:-445.741049437 time = 53.23s
[instance] Iteration 55,log-likehood:-446.035754358 time = 53.23s
[instance] Iteration 56,log-likehood:-446.258424984 time = 53.54s
[instance] Iteration 57,log-likehood:-445.922612659 time = 54.13s
[instance]

[instance] Iteration 171,log-likehood:-446.628547966 time = 53.00s
[instance] Iteration 172,log-likehood:-446.354746719 time = 52.88s
[instance] Iteration 173,log-likehood:-446.306227917 time = 53.30s
[instance] Iteration 174,log-likehood:-446.329843478 time = 53.24s
[instance] Iteration 175,log-likehood:-446.724596433 time = 52.82s
[instance] Iteration 176,log-likehood:-446.500438425 time = 52.99s
[instance] Iteration 177,log-likehood:-445.918264809 time = 52.91s
[instance] Iteration 178,log-likehood:-446.211564119 time = 53.19s
[instance] Iteration 179,log-likehood:-446.315465076 time = 52.58s
[instance] Iteration 180,log-likehood:-446.68488152 time = 52.63s
[instance] Iteration 181,log-likehood:-446.194573597 time = 52.64s
[instance] Iteration 182,log-likehood:-446.483956113 time = 52.71s
[instance] Iteration 183,log-likehood:-446.845440622 time = 52.48s
[instance] Iteration 184,log-likehood:-446.723790514 time = 52.84s
[instance] Iteration 185,log-likehood:-446.524290907 time = 52.

[instance] Iteration 92,log-likehood:-282.596593897 time = 51.93s
[instance] Iteration 93,log-likehood:-303.027018097 time = 50.99s
[instance] Iteration 94,log-likehood:-287.486485982 time = 50.91s
[instance] Iteration 95,log-likehood:-290.684982646 time = 50.76s
[instance] Iteration 96,log-likehood:-296.054313761 time = 50.99s
[instance] Iteration 97,log-likehood:-301.73701109 time = 51.30s
[instance] Iteration 98,log-likehood:-301.813633624 time = 50.53s
[instance] Iteration 99,log-likehood:-298.424599311 time = 50.36s
[instance] Iteration 100,log-likehood:-300.45363392 time = 50.18s
[instance] Iteration 101,log-likehood:-273.687621257 time = 50.45s
[instance] Iteration 102,log-likehood:-293.769171931 time = 50.12s
[instance] Iteration 103,log-likehood:-277.316949796 time = 50.66s
[instance] Iteration 104,log-likehood:-277.719558965 time = 50.45s
[instance] Iteration 105,log-likehood:-281.632529881 time = 50.43s
[instance] Iteration 106,log-likehood:-303.334566493 time = 50.34s
[inst

[instance] Iteration 15,log-likehood:-449.070435435 time = 121.58s
[instance] Iteration 16,log-likehood:-418.854862322 time = 132.81s
[instance] Iteration 17,log-likehood:-399.370029158 time = 125.19s
[instance] Iteration 18,log-likehood:-349.990748825 time = 129.08s
[instance] Iteration 19,log-likehood:-350.471715328 time = 121.15s
[instance] Iteration 20,log-likehood:-331.037319585 time = 122.19s
[instance] Iteration 21,log-likehood:-351.158281031 time = 120.26s
